In [184]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance
!pip install --upgrade mplfinance

Requirement already up-to-date: matplotlib in ./opt/anaconda3/lib/python3.8/site-packages (3.4.1)
Requirement already up-to-date: mplfinance in ./opt/anaconda3/lib/python3.8/site-packages (0.12.7a12)


In [185]:
import pandas as pd
import numpy as np

import time
import dateparser
import pytz
import json

import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

from mpl_finance import candlestick_ochl



from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from binance.client import Client

import datetime as dtt

In [186]:
df = pd.read_csv('btcusd-2015_2021_sp500_halving_gold.csv', index_col=0)

In [46]:
df

,Open Time,Open,High,Low,Close,Volume,NumberTransactions
0,2015-10-08,242.50,245.00,242.50,243.95,4595.840000,140348.0
1,2015-10-09,243.95,249.97,243.60,245.39,7651.630000,114644.0
2,2015-10-10,245.39,246.30,244.60,246.30,2984.440000,127005.0
3,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550.0
4,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410.0
...,...,...,...,...,...,...,...
1839,2021-04-15,62959.53,63800.00,62020.00,63159.98,51649.700340,1938028.0
1840,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0
1841,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816.0
1842,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089.0


In [187]:
df.describe()

,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold
count,2019.000000,2019.000000,2019.000000,2019.000000,2.019000e+03,2.019000e+03,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000
mean,8118.617256,8373.885953,7845.329911,8146.388430,1.738177e+07,4.597203e+05,2722.603843,2723.500594,1.340763,1448.567221,1448.691441
std,10787.222476,11161.188328,10387.891407,10839.617374,3.437011e+07,5.251333e+05,506.494218,507.698068,0.752121,210.520585,210.443203
min,246.300000,249.500000,236.000000,247.600000,0.000000e+00,1.045500e+05,1829.080000,1833.400000,1.000000,1070.800000,1070.100000
25%,1049.880000,1082.560000,1022.655000,1054.860000,3.166145e+05,2.237780e+05,2341.890000,2342.610000,1.000000,1310.400000,1310.400000
50%,6410.010000,6546.290000,6320.010000,6412.000000,5.270744e+06,2.838960e+05,2717.070000,2717.350000,1.000000,1369.800000,1370.000000
75%,9479.285000,9699.530000,9231.755000,9484.275000,2.003407e+07,3.529245e+05,2986.200000,2988.865000,1.000000,1541.300000,1540.400000
max,63575.010000,64854.000000,62020.000000,63575.000000,5.606539e+08,4.431451e+06,4185.470000,4175.600000,3.000000,2069.400000,2076.400000


In [190]:
df.dtypes

Open Time             datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close                        float64
Volume                       float64
NumberTransactions           float64
Close SP500                  float64
Open SP500                   float64
halving                        int64
Close Gold                   float64
Open Gold                    float64
dtype: object

In [189]:
df['Open Time'] = pd.to_datetime(df['Open Time'])

In [142]:
print(df)

      Open Time      Open      High       Low     Close         Volume  \
0    2015-10-11    246.30    249.50    246.30    249.50    3021.120000   
1    2015-10-12    249.50    249.50    247.60    247.60    9493.890000   
2    2015-10-13    247.60    252.87    245.75    250.00   12875.400000   
3    2015-10-14    250.00    254.75    250.00    253.00   30115.770000   
4    2015-10-15    253.00    258.98    253.00    255.26   50535.450000   
...         ...       ...       ...       ...       ...            ...   
2014 2021-04-16  63158.74  63520.61  60000.00  61334.80   91764.139884   
2015 2021-04-17  61334.81  62506.05  59580.91  60006.66   58912.256128   
2016 2021-04-18  60006.67  60499.00  50931.30  56150.01  124882.131824   
2017 2021-04-19  56150.01  57526.81  54221.58  55633.14   78229.042267   
2018 2021-04-20  55633.14  56599.53  53329.96  56566.46   60500.152593   

      NumberTransactions  Close SP500  Open SP500  halving  Close Gold  \
0               104550.0      2081.72

In [51]:
sp500 = pd.read_csv("Datos históricos S&P 500.csv")
sp500.head()

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
0,19.04.2021,"4.162,80","4.175,60","4.180,02","4.150,54",-,"-0,54%"
1,16.04.2021,"4.185,47","4.174,14","4.191,31","4.170,75",-,"0,36%"
2,15.04.2021,"4.170,42","4.139,76","4.173,49","4.139,76",-,"1,11%"
3,14.04.2021,"4.124,66","4.141,58","4.151,69","4.120,87",-,"-0,41%"
4,13.04.2021,"4.141,59","4.130,10","4.148,00","4.124,43",-,"0,33%"


In [100]:
sp500['Open Time'] = pd.to_datetime(sp500['Fecha'])
sp500['OpenSP'] = sp500['Apertura']
sp500['CloseSP'] = sp500['Último']

In [101]:
df_with_sp500 = df.merge(sp500[["OpenSP", "CloseSP", "Open Time"]], on="Open Time")

In [102]:
df_with_sp500.tail()

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,OpenSP,CloseSP
1222,2021-04-13,59860.01,63777.77,59805.15,63575.00,82848.688746,2703735.0,"4.130,10","4.141,59"
1223,2021-04-14,63575.01,64854.00,61301.00,62959.53,82616.343993,2645625.0,"4.141,58","4.124,66"
1224,2021-04-15,62959.53,63800.00,62020.00,63159.98,51649.700340,1938028.0,"4.139,76","4.170,42"
1225,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0,"4.174,14","4.185,47"
1226,2021-04-19,56150.01,57526.81,54221.58,54707.06,63892.980917,1928047.0,"4.175,60","4.162,80"


In [56]:
df_with_sp500.head()

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Último,Apertura
0,2015-10-11,246.3,249.50,246.30,249.50,3021.12,104550.0,"2.081,72","2.077,19"
1,2015-10-12,249.5,249.50,247.60,247.60,9493.89,134410.0,"2.052,23","2.047,93"
2,2015-10-13,247.6,252.87,245.75,250.00,12875.40,134878.0,"2.003,69","2.015,00"
3,2015-10-14,250.0,254.75,250.00,253.00,30115.77,141985.0,"1.994,24","2.003,66"
4,2015-10-15,253.0,258.98,253.00,255.26,50535.45,153495.0,"2.023,86","1.996,47"


In [69]:
df_with_sp500.corr()

,Open,High,Low,Close,Volume,NumberTransactions,Último,Apertura
Open,1.000000,0.999264,0.998511,0.998356,0.080261,0.844734,0.781894,0.783142
High,0.999264,1.000000,0.998498,0.999243,0.086268,0.849991,0.780992,0.782148
Low,0.998511,0.998498,1.000000,0.999083,0.061633,0.839640,0.784356,0.785526
Close,0.998356,0.999243,0.999083,1.000000,0.076407,0.846050,0.782845,0.783863
Volume,0.080261,0.086268,0.061633,0.076407,1.000000,-0.127307,0.037086,0.037549
NumberTransactions,0.844734,0.849991,0.839640,0.846050,-0.127307,1.000000,0.669738,0.671507
Último,0.781894,0.780992,0.784356,0.782845,0.037086,0.669738,1.000000,0.998841
Apertura,0.783142,0.782148,0.785526,0.783863,0.037549,0.671507,0.998841,1.000000


In [68]:
df_with_sp500.dtypes

Open Time             datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close                        float64
Volume                       float64
NumberTransactions           float64
Último                       float64
Apertura                     float64
dtype: object

In [103]:
df_with_sp500["CloseSP"] = df_with_sp500["CloseSP"].apply(lambda x: x.replace(".", ""))
df_with_sp500["CloseSP"] = df_with_sp500["CloseSP"].apply(lambda x: x.replace(",", "."))

In [104]:
df_with_sp500["OpenSP"] = df_with_sp500["OpenSP"].apply(lambda x: x.replace(".", ""))
df_with_sp500["OpenSP"] = df_with_sp500["OpenSP"].apply(lambda x: x.replace(",", "."))

In [105]:
df_with_sp500["CloseSP"] = df_with_sp500["CloseSP"].astype(float)
df_with_sp500["OpenSP"] = df_with_sp500["OpenSP"].astype(float)

In [106]:
df_with_sp500.columns.values

array(['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
       'NumberTransactions', 'OpenSP', 'CloseSP'], dtype=object)

In [107]:
df_with_sp500.head()

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,OpenSP,CloseSP
0,2015-10-11,246.3,249.50,246.30,249.50,3021.12,104550.0,2077.19,2081.72
1,2015-10-12,249.5,249.50,247.60,247.60,9493.89,134410.0,2047.93,2052.23
2,2015-10-13,247.6,252.87,245.75,250.00,12875.40,134878.0,2015.00,2003.69
3,2015-10-14,250.0,254.75,250.00,253.00,30115.77,141985.0,2003.66,1994.24
4,2015-10-15,253.0,258.98,253.00,255.26,50535.45,153495.0,1996.47,2023.86


In [108]:
df_with_sp500.tail()

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,OpenSP,CloseSP
1222,2021-04-13,59860.01,63777.77,59805.15,63575.00,82848.688746,2703735.0,4130.10,4141.59
1223,2021-04-14,63575.01,64854.00,61301.00,62959.53,82616.343993,2645625.0,4141.58,4124.66
1224,2021-04-15,62959.53,63800.00,62020.00,63159.98,51649.700340,1938028.0,4139.76,4170.42
1225,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0,4174.14,4185.47
1226,2021-04-19,56150.01,57526.81,54221.58,54707.06,63892.980917,1928047.0,4175.60,4162.80


In [191]:
DowJones = pd.read_csv('Datos históricos Dow Jones Industrial Average.csv')

In [196]:
DowJones.head()

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.,Open Time,OpenDJ,CloseDJ
0,19.04.2021,"34.077,63","34.182,38","34.182,38","33.980,56","315,62M","-0,36%",2021-04-19,"34.182,38","34.077,63"
1,16.04.2021,"34.200,67","34.058,16","34.256,75","34.058,16","348,18M","0,48%",2021-04-16,"34.058,16","34.200,67"
2,15.04.2021,"34.035,99","33.788,53","34.068,73","33.788,53","321,83M","0,90%",2021-04-15,"33.788,53","34.035,99"
3,14.04.2021,"33.730,89","33.668,95","33.911,25","33.668,95","318,67M","0,16%",2021-04-14,"33.668,95","33.730,89"
4,13.04.2021,"33.677,27","33.721,16","33.741,64","33.545,84","330,00M","-0,20%",2021-04-13,"33.721,16","33.677,27"


In [195]:
DowJones.columns.values

array(['Fecha', 'Último', 'Apertura', 'Máximo', 'Mínimo', 'Vol.',
       '% var.', 'Open Time', 'OpenDJ', 'CloseDJ'], dtype=object)

In [193]:
DowJones['Open Time'] = pd.to_datetime(DowJones['Fecha'])

In [194]:
DowJones['OpenDJ'] = DowJones['Apertura']
DowJones['CloseDJ'] = DowJones['Último']

In [179]:
DowJones['OpenDJ']

0       34.182,38
1       34.058,16
2       33.788,53
3       33.668,95
4       33.721,16
          ...    
1384    17.141,75
1385    16.944,86
1386    17.079,08
1387    17.113,55
1388    17.082,29
Name: OpenDJ, Length: 1389, dtype: object

In [197]:
df_with_indexes = df.merge(DowJones[["OpenDJ", "CloseDJ", "Open Time"]], how='left', on="Open Time")

In [198]:
df_with_indexes.dtypes

Open Time             datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close                        float64
Volume                       float64
NumberTransactions           float64
Close SP500                  float64
Open SP500                   float64
halving                        int64
Close Gold                   float64
Open Gold                    float64
OpenDJ                        object
CloseDJ                       object
dtype: object

In [199]:
df_with_indexes.fillna(method='ffill', inplace=True)

In [202]:
df_with_indexes

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ
0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550.0,2081.72,2077.19,1,1149.4,1149.4,"17.724,13","17.758,21"
1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410.0,2052.23,2047.93,1,1130.3,1130.3,"17.493,17","17.574,75"
2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878.0,2003.69,2015.00,1,1231.7,1231.7,"17.113,55","17.081,89"
3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985.0,1994.24,2003.66,1,1243.6,1243.6,"17.079,08","16.924,75"
4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495.0,2023.86,1996.47,1,1251.4,1251.4,"16.944,86","17.141,75"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0,4185.47,4174.14,3,1779.0,1770.4,"34.058,16","34.200,67"
2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816.0,4185.47,4174.14,3,1779.0,1770.4,"34.058,16","34.200,67"
2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089.0,4185.47,4174.14,3,1779.0,1770.4,"34.058,16","34.200,67"
2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527.0,4162.80,4175.60,3,1769.4,1782.1,"34.182,38","34.077,63"


In [203]:
df_with_indexes["CloseDJ"] = df_with_indexes["CloseDJ"].apply(lambda x: x.replace(".", ""))
df_with_indexes["CloseDJ"] = df_with_indexes["CloseDJ"].apply(lambda x: x.replace(",", "."))

In [204]:
df_with_indexes["OpenDJ"] = df_with_indexes["OpenDJ"].apply(lambda x: x.replace(".", ""))
df_with_indexes["OpenDJ"] = df_with_indexes["OpenDJ"].apply(lambda x: x.replace(",", "."))

In [205]:
df_with_indexes["CloseDJ"] = df_with_indexes["CloseDJ"].astype(float)
df_with_indexes["OpenDJ"] = df_with_indexes["OpenDJ"].astype(float)

In [206]:
df_with_indexes

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ
0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550.0,2081.72,2077.19,1,1149.4,1149.4,17724.13,17758.21
1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410.0,2052.23,2047.93,1,1130.3,1130.3,17493.17,17574.75
2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878.0,2003.69,2015.00,1,1231.7,1231.7,17113.55,17081.89
3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985.0,1994.24,2003.66,1,1243.6,1243.6,17079.08,16924.75
4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495.0,2023.86,1996.47,1,1251.4,1251.4,16944.86,17141.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67
2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67
2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67
2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527.0,4162.80,4175.60,3,1769.4,1782.1,34182.38,34077.63


In [120]:
df_with_indexes.corr()

,Open,High,Low,Close,Volume,NumberTransactions,OpenSP,CloseSP,OpenDJ,CloseDJ
Open,1.000000,0.999264,0.998511,0.998356,0.080261,0.844734,0.783142,0.781894,0.705910,0.704959
High,0.999264,1.000000,0.998498,0.999243,0.086268,0.849991,0.782148,0.780992,0.704193,0.703305
Low,0.998511,0.998498,1.000000,0.999083,0.061633,0.839640,0.785526,0.784356,0.707785,0.706904
Close,0.998356,0.999243,0.999083,1.000000,0.076407,0.846050,0.783863,0.782845,0.705694,0.704917
Volume,0.080261,0.086268,0.061633,0.076407,1.000000,-0.127307,0.037549,0.037086,0.123480,0.122968
NumberTransactions,0.844734,0.849991,0.839640,0.846050,-0.127307,1.000000,0.671507,0.669738,0.541122,0.539072
OpenSP,0.783142,0.782148,0.785526,0.783863,0.037549,0.671507,1.000000,0.998841,0.972595,0.971329
CloseSP,0.781894,0.780992,0.784356,0.782845,0.037086,0.669738,0.998841,1.000000,0.971643,0.972670
OpenDJ,0.705910,0.704193,0.707785,0.705694,0.123480,0.541122,0.972595,0.971643,1.000000,0.998752
CloseDJ,0.704959,0.703305,0.706904,0.704917,0.122968,0.539072,0.971329,0.972670,0.998752,1.000000


In [207]:
df_with_indexes[df_with_indexes['Open Time']=='2020-04-09']

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ
1642,2020-04-09,7369.6,7378.85,7115.04,7315.25,16310014.53,286706.0,3426.96,3453.6,1,1934.3,1938.0,28341.05,28133.31


In [208]:
df_with_indexes[df_with_indexes['Open Time']=='2020-04-15']

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ
1648,2020-04-15,6868.57,6933.0,6605.0,6621.24,61571.384994,891627.0,2783.36,2795.64,1,1744.7,1760.7,23600.72,23504.35


In [209]:
df[df['Open Time']=='2020-04-13']

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold
1646,2020-04-13,6903.79,6903.79,6575.0,6837.91,96415.476573,1092292.0,2761.63,2782.46,1,1762.7,1741.6


In [133]:
df.describe()

,Open,High,Low,Close,Volume,NumberTransactions
count,1844.000000,1844.000000,1844.000000,1844.000000,1.844000e+03,1.844000e+03
mean,7920.777847,8181.400033,7643.250098,7948.408568,1.902526e+07,4.248306e+05
std,11216.782107,11610.123416,10800.013665,11268.061827,3.552879e+07,5.279225e+05
min,242.500000,245.000000,236.000000,243.950000,0.000000e+00,1.045500e+05
25%,907.790000,923.350000,889.322500,911.740000,6.218876e+05,2.195405e+05
50%,5705.155000,5893.985000,5511.955000,5722.655000,7.082184e+06,2.728605e+05
75%,8903.045000,9180.260000,8661.252500,8903.045000,2.193622e+07,3.307325e+05
max,63575.010000,64854.000000,62020.000000,63575.000000,5.606539e+08,4.431451e+06


In [216]:
df_with_indexes.to_csv('df_indexes_gold.csv')

## Standard Deviation

In [212]:
def standart_deviation(df, n):
  df = df.join(pd.Series(df['Close'].rolling(n, min_periods=n).std(), name='STD_' + str(n)))
  return df

In [214]:
df = df_with_indexes.copy()

In [220]:
standart_deviation(df,100)

,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ,STD_100
0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550.0,2081.72,2077.19,1,1149.4,1149.4,17724.13,17758.21,NaN
1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410.0,2052.23,2047.93,1,1130.3,1130.3,17493.17,17574.75,NaN
2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878.0,2003.69,2015.00,1,1231.7,1231.7,17113.55,17081.89,NaN
3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985.0,1994.24,2003.66,1,1243.6,1243.6,17079.08,16924.75,NaN
4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495.0,2023.86,1996.47,1,1251.4,1251.4,16944.86,17141.75,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67,9824.172875
2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67,9852.549571
2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67,9849.510266
2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527.0,4162.80,4175.60,3,1769.4,1782.1,34182.38,34077.63,9835.923277
